# W1 [pbatoolkit](https://pypi.org/project/pbatoolkit/)'s FEM module

This first workshop on FEM aims to help the reader:
- explore/analyze FEM quantities/operators
- familiarize themself with our [Physics Based Animation Toolkit](https://github.com/Q-Minh/PhysicsBasedAnimationToolkit)'s Python interface

Start by installing the required dependencies by running `pip install -r requirements.txt` on the command line. FEM relies on having meshed geometries at hand, hence we recommend finding clean surface mesh geometry and then using our [mesh processing scripts](/python/tools/mesh/) to tetrahedralize them, if you don't already have access to meshes. You can download surface meshes on [Thingi10K](https://ten-thousand-models.appspot.com/), [TurboSquid](https://www.turbosquid.com/) or other.

If you are interested in profiling calls to FEM operations, also download the [Tracy profiling server](https://github.com/wolfpld/tracy/releases/tag/v0.10), run `tracy.exe` and connect to this process.

## Loading input mesh geometry

Use [meshio](https://pypi.org/project/meshio/) to load meshes from disk to memory, and render them using [polyscope](https://pypi.org/project/polyscope/).

## Constructing FEM meshes

From the input mesh geometry, construct an FEM mesh using `pbatoolkit`. Start by exploring the API of the FEM module of `pbatoolkit`. Then, try constructing meshes of different shape function orders on either of line, triangle, and/or tetrahedral meshes. You can do the same for regular geometry (rectangles and boxes) if you want.

Use Python's `help` command on `pbatoolkit`'s mesh instance, and check that you can access its nodes' positions, its elements' nodal indices, its dimensionality, element type, etc.

> Note that the FEM mesh and the geometric mesh do NOT share the same element indices. Both the FEM and geometric mesh have the same number of elements, and they are ordered in exactly the same way, but the geometric mesh's array of vertex positions does not necessarily correspond to the FEM mesh's array of node positions.

Keep an eye on how the number of FEM nodes changes when the mesh's shape function order changes, and the computational cost of constructing higher order meshes. Visualize the FEM mesh's nodes on top of the geometric mesh using `polyscope`.

In [ ]:
# Import modules of interest
import pbatoolkit as pbat
import pbatoolkit.fem, pbatoolkit.profiling
# Consult FEM module's API
help(pbat.fem)

## Mass matrix

1. Construct the mass matrix using `pbatoolkit`'s built-in `mass_matrix` function, and then using only the shape function and quadrature matrices, and compare the resulting mass matrices. Call `to_matrix` to get a `scipy` sparse matrix from `pbatoolkit`'s mass matrix instance.
2. Analyze its properties. Is it square, symmetric, positive definite? What is its sparsity? 
3. What happens if you use the correct quadrature order, a lower quadrature order, and a higher quadrature order? 
4. How computationally costly is mass matrix construction in both cases?
5. Use the mass matrix's `apply` method on some arbitrary input vector (or matrix) of appropriate dimensions. How costly is this matrix-free multiplication?

## Load vector

1. Construct a piecewise constant load vector using `pbatoolkit`'s built-in `load_vector` function, and then using only the shape function and quadrature matrices. You can use some common forcing function, like gravity, i.e. $f_e(X) = [0, 0, -9.81]$. Compare the resulting discrete load vectors.
2. For a higher-order mesh, is there any difference when using different quadrature orders in both cases? Why?
3. How computationally costly is its construction in both cases?

## Laplacian matrix
 1. Construct the Laplacian matrix using `pbatoolkit`'s built-in `laplacian_matrix` function, and then using only the shape gradient and quadrature matrices, and compare the resulting Laplacians.
 2. Analyze its properties, similar to the mass matrix case.
 3. Try varying the quadrature orders and evaluate the consequences.
 4. What is the computational cost of its different construction approaches?
 5. Use the Laplacian matrix's `apply` method similar to the mass matrix case.

## Solutions

Scripts in the [examples](/python/examples/) folder show different ways to construct relevant FEM quantities using `pbatoolkit` for different mesh types, different shape function orders, and different construction approaches.